In [2]:
import PyPDF2
import PySimpleGUI as sg
from pathlib import Path
import logging
import re
import sys, getopt


# TO DO
* [x] wrap **everything** in a `try`
* [ ] add gui
* [ ] test against MAP results
* [x] add original filename to output
* [ ] pyinstaller

In [3]:
def get_breaks(file):
    file = Path(file).expanduser().absolute()
    
    try:
        source_pdf = PyPDF2.PdfFileReader(str(file), False)
    except (FileNotFoundError, PyPDF2.utils.PdfReadError, IsADirectoryError) as e:
        logging.error(f'Failed to read source pdf: {e}')
        return []
    
    page_breaks = []
    
    last_record = {'id': None, 'pages': []}
    
    for page in range(0, source_pdf.numPages):
        this_record = ({'id': None, 'pages': []})
        text = source_pdf.getPage(page).extractText()
        student_number = re.search('student\W?id:\W?(\d{5,})', text, re.IGNORECASE)
        
        if student_number:
            page_breaks.append(last_record)
            this_record['id'] = student_number.group(1)
            this_record['pages'].append(page)
            last_record = this_record
            
        else:
            last_record['pages'].append(page)
            
    return page_breaks
            
        

In [4]:
def split_pdf(file, page_breaks):
    file = Path(file).expanduser().absolute()
    dest_path = file.parent/f'{file.name}_split'
    
    output = {}
    
    try:
        source_pdf = PyPDF2.PdfFileReader(str(file))
    except (FileNotFoundError, PyPDF2.utils.PdfReadError, IsADirectoryError) as e:
        logging.error(f'Failed to split source pdf: {e}')
        return False
    
    for record in page_breaks:
        output_path = 'Failed'
        if not record['id']:
            continue
        pdf_writer = PyPDF2.PdfFileWriter()
        for page in record['pages']:
            try:
                pdf_writer.addPage(source_pdf.getPage(page))
            except Exception as e:
                logging.error(f'failed to add page: "{page}" for record: "{record}" error: {e}')
                
            
        
        try:
            output_path = dest_path/f'{record["id"]}.pdf'
            output_path.parent.mkdir(exist_ok=True)
            with open(str(output_path), "wb") as dest_file:
                pdf_writer.write(dest_file)

        except Exception as e:
            logging.error(f'error outputing pdf: {e}')
            output_path = f'Failed outputing pdf: {e}'
            pass
        finally:
            output[record['id']] = output_path
    return output



In [10]:
def main():
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    interactive = False
    
    if (len(sys.argv) == 1) or ('-f' in sys.argv):
        interactive = True
     
    else:
        fname = sys.argv[1]

    while interactive:
        logging.debug('prompt user for PDF file')
        fname = sg.popup_get_file('Choose a PDF to split -- "Cancel" to quit')
        logging.debug(f'fname returned: {fname}')
        if fname in [None, ]:
            logging.info('user canceled')
            break
        if fname:
            breaks = get_breaks(fname)
            output = split_pdf(fname, breaks)
            
        if output:
            ## add a popup that summarizes what was done
            print(output)
        else:
            sg.popup('Cancel', "No PDF supplied")


In [12]:
if __name__ == "__main__":
    main()

DEBUG:root:prompt user for PDF file
DEBUG:root:fname returned: /Users/aciuffo/Documents/src/pyPDF_split/data/UE_Semester2_ReportCard_2019-2020_Gr3.pdf
DEBUG:root:prompt user for PDF file


{'505341': PosixPath('/Users/aciuffo/Documents/src/pyPDF_split/data/UE_Semester2_ReportCard_2019-2020_Gr3.pdf_split/505341.pdf'), '505464': PosixPath('/Users/aciuffo/Documents/src/pyPDF_split/data/UE_Semester2_ReportCard_2019-2020_Gr3.pdf_split/505464.pdf'), '504864': PosixPath('/Users/aciuffo/Documents/src/pyPDF_split/data/UE_Semester2_ReportCard_2019-2020_Gr3.pdf_split/504864.pdf'), '505557': PosixPath('/Users/aciuffo/Documents/src/pyPDF_split/data/UE_Semester2_ReportCard_2019-2020_Gr3.pdf_split/505557.pdf'), '503509': PosixPath('/Users/aciuffo/Documents/src/pyPDF_split/data/UE_Semester2_ReportCard_2019-2020_Gr3.pdf_split/503509.pdf'), '503845': PosixPath('/Users/aciuffo/Documents/src/pyPDF_split/data/UE_Semester2_ReportCard_2019-2020_Gr3.pdf_split/503845.pdf'), '504450': PosixPath('/Users/aciuffo/Documents/src/pyPDF_split/data/UE_Semester2_ReportCard_2019-2020_Gr3.pdf_split/504450.pdf'), '503835': PosixPath('/Users/aciuffo/Documents/src/pyPDF_split/data/UE_Semester2_ReportCard_2019-

DEBUG:root:fname returned: None
INFO:root:user canceled


In [ ]:
'-f' in sys.argv

In [ ]:
sys.argv